In [20]:
import pandas as pd
import numpy as np

### Read data

In [3]:
drivers = pd.read_csv("../data/drivers.csv")
pings = pd.read_csv("../data/pings.csv")

In [3]:
pings.head()

,driver_id,ping_timestamp
0,899313,1496278800
1,373017,1496278800
2,798984,1496278800
3,245966,1496278800
4,689783,1496278800


### Convert to datetime, extract date

In [4]:
pings['datetime'] = pd.to_datetime(pings.ping_timestamp, unit='s')
pings['date'] = pd.to_datetime(pings.datetime).dt.date

In [5]:
pings.tail()

,driver_id,ping_timestamp,datetime,date
50528696,282579,1498100130,2017-06-22 02:55:30,2017-06-22
50528697,282579,1498100145,2017-06-22 02:55:45,2017-06-22
50528698,282579,1498100160,2017-06-22 02:56:00,2017-06-22
50528699,282579,1498100190,2017-06-22 02:56:30,2017-06-22
50528700,282579,1498100220,2017-06-22 02:57:00,2017-06-22


### For each driver, for each day, get total number of hours spent online
- For each day, for each driver, get the min(time) and max(time) of the day.
- Example: date, driver_id, min(datetime), max(datetime)

In [47]:
# Do a group by to get the min and max timestamp by driver, day first
pings_minmax = pings.groupby(['driver_id','date'], as_index=False).agg({'datetime':[min,max]})

In [48]:
# rename columns
pings_minmax.columns = ['driver_id','date','start_time','end_time']

In [49]:
# Next, get the time difference from the min and max timestamp columns for each driver for each day
pings_minmax['time_online'] = pings_minmax['end_time'] - pings_minmax['start_time']

In [60]:
# Get in terms of hours the driver is online (this method uses numpy)
pings_minmax['hours_online_float'] = pings_minmax['time_online']/np.timedelta64(1,'h')

In [61]:
# Get in terms of hours the driver is online (this method uses pandas)
pings_minmax['hours_online_int'] = pd.to_datetime(pings_minmax['time_online']).dt.hour

In [58]:
pings_minmax.head()

,driver_id,date,start_time,end_time,time_online,hours_online_int,hours_online_float
0,111556,2017-06-01,2017-06-01 01:09:00,2017-06-01 03:22:00,02:13:00,2,2.216667
1,111556,2017-06-02,2017-06-02 01:22:00,2017-06-02 03:52:00,02:30:00,2,2.500000
2,111556,2017-06-05,2017-06-05 01:37:00,2017-06-05 06:17:00,04:40:00,4,4.666667
3,111556,2017-06-06,2017-06-06 01:51:00,2017-06-06 04:58:00,03:07:00,3,3.116667
4,111556,2017-06-07,2017-06-07 01:22:15,2017-06-07 04:00:00,02:37:45,2,2.629167


In [ ]:
pings_minmax.describe()

In [39]:
pings_minmax.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41676 entries, 0 to 41675
Data columns (total 6 columns):
driver_id      41676 non-null int64
date           41676 non-null object
start_time     41676 non-null datetime64[ns]
end_time       41676 non-null datetime64[ns]
time_online    41676 non-null float64
time_diff      41676 non-null timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(1), timedelta64[ns](1)
memory usage: 2.2+ MB
